In [1]:
%load_ext autoreload
%autoreload 2

from nclustRL.trainer import Trainer
from ray.rllib.agents.ppo import PPOTrainer

Using backend: pytorch


In [2]:
trainer = Trainer(
    trainer=PPOTrainer,
    env='BiclusterEnv-v0',
    save_dir='/home/pedro.cotovio/Repos/nclustRL/Exp/Exp1'
)

2021-12-30 11:23:48,562	WARNING ppo.py:143 -- `train_batch_size` (1024) cannot be achieved with your other settings (num_workers=3 num_envs_per_worker=1 rollout_fragment_length=341)! Auto-adjusting `rollout_fragment_length` to 341.
2021-12-30 11:23:48,562	INFO ppo.py:166 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(RolloutWorker pid=20333) Using backend: pytorch
(RolloutWorker pid=20329) Using backend: pytorch
(RolloutWorker pid=20331) Using backend: pytorch
(RolloutWorker pid=20333) 2021-12-30 11:23:53,682	INFO worker.py:852 -- Calling ray.init() again after it has already been called.
(RolloutWorker pid=20331) 2021-12-30 11:23:53,797	INFO worker.py:852 -- Calling ray.init() again after it has already been called.
(RolloutWorker pid=20329) 2021-12-30 11:23:53,951	INFO worker.py:852 -- Calling ray.init() again after it has already been called.
(RolloutWorker pid=20333) 2

In [5]:
trainer.agent.train()

(RolloutWorker pid=20331) 2021-12-30 11:28:39,890	INFO sampler.py:1050 -- Inputs to compute_actions():
(RolloutWorker pid=20331) 
(RolloutWorker pid=20331) { 'default_policy': [ { 'data': { 'agent_id': 'agent0',
(RolloutWorker pid=20331)                                   'env_id': 0,
(RolloutWorker pid=20331)                                   'info': {},
(RolloutWorker pid=20331)                                   'obs': { 'action_mask': np.ndarray((4,), dtype=float32, min=0.0, max=1.0, mean=0.5),
(RolloutWorker pid=20331)                                            'avail_actions': np.ndarray((4,), dtype=float32, min=1.0, max=1.0, mean=1.0),
(RolloutWorker pid=20331)                                            'state': Graph(num_nodes={'col': 10, 'row': 100},
(RolloutWorker pid=20331)       num_edges={('row', 'elem', 'col'): 1000},
(RolloutWorker pid=20331)       metagraph=[('row', 'col', 'elem')])},
(RolloutWorker pid=20331)                                   'prev_action': np.ndarray((1

{'episode_reward_max': -1.8455309222842375,
 'episode_reward_min': -1.8671428422190117,
 'episode_reward_mean': -1.8566294914913752,
 'episode_len_mean': 1001.0,
 'episode_media': {},
 'episodes_this_iter': 3,
 'policy_reward_min': {},
 'policy_reward_max': {},
 'policy_reward_mean': {},
 'custom_metrics': {},
 'hist_stats': {'episode_reward': [-1.8457294917204952,
   -1.8506421356712597,
   -1.8455309222842375,
   -1.8652934214785981,
   -1.8654381355746479,
   -1.8671428422190117],
  'episode_lengths': [1001, 1001, 1001, 1001, 1001, 1001]},
 'sampler_perf': {'mean_raw_obs_processing_ms': 0.25547610235272056,
  'mean_inference_ms': 8.731565234282522,
  'mean_action_processing_ms': 0.2235897798484958,
  'mean_env_wait_ms': 16.234337814841346,
  'mean_env_render_ms': 0.0},
 'off_policy_estimator': {},
 'num_healthy_workers': 3,
 'timesteps_total': 6138,
 'timesteps_this_iter': 0,
 'agent_timesteps_total': 6138,
 'timers': {'sample_time_ms': 94623.896,
  'sample_throughput': 21.622,
  'l

## Test Model

In [13]:
from nclustRL.models import RlModel, GraphEncoder
from nclustRL.models.model import FcNet
from nclustRL.utils.helper import generate_dummy_obs, transform_obs
import nclustenv
from nclustenv.configs.biclustering.binary import base
from nclustRL.configs.default_configs import MODEL_DEFAULTS
from gym.wrappers import TransformObservation

import torch as th 
from ray.rllib.utils.torch_utils import FLOAT_MIN, FLOAT_MAX

BATCH = 32

def make_env():
    return TransformObservation(nclustenv.make('BiclusterEnv-v0', **base), transform_obs)

def init_rlmodel():

    env = make_env()
    
    model = RlModel(env.observation_space, env.action_space, 28, MODEL_DEFAULTS, '')
    return model.cuda()

def generate_dummy_dict():
    env = make_env()
    return {'obs': [env.reset() for _ in range(BATCH)]}

rlmodel = init_rlmodel()

In [14]:
env = make_env()

model = GraphEncoder(env.observation_space['state'].n, [64], ['elem'])
model.cuda()
obs_flat = rlmodel._generate_dummy_obs(th.zeros((BATCH, 2)))
hg = model(obs_flat)
hg.size()

/home/pedro.cotovio/Repos/nclustRL/nclustRL/models/model.py:144: UserWarning: Forward is running with random obs, this should only be acceptable during policy inicialization.
  warnings.warn('Forward is running with random obs, this should only be acceptable during policy inicialization.')


torch.Size([32, 64])

In [7]:
model2 = FcNet(64, [256, 256])
model2.cuda()
logits = model2(hg)
logits.size()

torch.Size([32, 256])

In [8]:
_params = rlmodel._param_branch(logits)
_params.size()

torch.Size([32, 24])

In [9]:
_actions = rlmodel._action_branch(logits)
_actions.size()

torch.Size([32, 4])

In [10]:
rlmodel._value_branch(logits).squeeze(1).size()

torch.Size([32])

In [11]:
obs = env.reset()

avail_actions = th.tensor(obs["avail_actions"], device=obs_flat.device).unsqueeze(0).repeat(32, 1)
action_mask = th.tensor(obs["action_mask"], device=obs_flat.device).unsqueeze(0).repeat(32, 1)

intent_vector = th.unsqueeze(_actions, 1)
action_logits = th.sum(avail_actions * intent_vector, dim=1)
inf_mask = th.clamp(th.log(action_mask), FLOAT_MIN, FLOAT_MAX)

th.cat(((action_logits + inf_mask), _params), dim=1).size()

torch.Size([32, 28])

In [25]:
import numpy as np
import dgl
dgl.batch(list(np.array([env.reset() for _ in range(10)])))

AttributeError: 'dict' object has no attribute 'is_block'

In [35]:
dgl.batch(list(np.array([env.reset()['state'] for _ in range(10)])))

Graph(num_nodes={'col': 100, 'row': 1000},
      num_edges={('row', 'elem', 'col'): 10000},
      metagraph=[('row', 'col', 'elem')])